In [1]:
import pandas as pd
import numpy as np
import sys

import warnings
import math

df=pd.read_csv('../data/data.csv.gz', compression='gzip', sep=',')
df['timestamp'] = pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_convert('Europe/Stockholm')


In [2]:
stock_delta = df['stock_id'] - df['stock_id'].shift()
time_delta = (df['timestamp'] - df['timestamp'].shift()).fillna(10000).dt.total_seconds()

/home/ken/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  


In [3]:
stock_split_index=df.index[stock_delta!=0].tolist()
stock_split_index.append(len(df))

In [4]:
day_split_index=df.index[abs(time_delta)>36000].tolist()
day_split_index.insert(0, 0)
day_split_index.append(len(df))

In [7]:
stock_id_list = df['stock_id'].unique().tolist()
i = 0
for stock_index in stock_id_list:
    
    print("index: {}, id:{}".format(i, stock_index))
    i+=1

index: 0, id:101
index: 1, id:1027
index: 2, id:120
index: 3, id:139301
index: 4, id:15285
index: 5, id:160271
index: 6, id:161
index: 7, id:18634
index: 8, id:281
index: 9, id:283
index: 10, id:285
index: 11, id:300
index: 12, id:323
index: 13, id:340
index: 14, id:3524
index: 15, id:361
index: 16, id:366
index: 17, id:3966
index: 18, id:401
index: 19, id:402
index: 20, id:45
index: 21, id:46
index: 22, id:47
index: 23, id:4928
index: 24, id:5095
index: 25, id:81
index: 26, id:812
index: 27, id:819
index: 28, id:992
index: 29, id:999


In [6]:
current_stock_index = 0
time_series_list = []
time_series_all_list = []
for i in range(len(day_split_index) - 1):
    start_index = day_split_index[i]
    end_index = day_split_index[i + 1]
    day_time_series = df.iloc[start_index:end_index]
    time_series_list.append(day_time_series)
    if end_index == stock_split_index[current_stock_index + 1]:
        print("finished stock: {}, got {} time series".format(current_stock_index, len(time_series_list)))
        time_series_all_list.append(time_series_list)
        time_series_list = []
        current_stock_index += 1

finished stock: 0, got 53 time series
finished stock: 1, got 53 time series
finished stock: 2, got 53 time series
finished stock: 3, got 53 time series
finished stock: 4, got 53 time series
finished stock: 5, got 53 time series
finished stock: 6, got 53 time series
finished stock: 7, got 53 time series
finished stock: 8, got 53 time series
finished stock: 9, got 53 time series
finished stock: 10, got 53 time series
finished stock: 11, got 53 time series
finished stock: 12, got 53 time series
finished stock: 13, got 53 time series
finished stock: 14, got 53 time series
finished stock: 15, got 53 time series
finished stock: 16, got 53 time series
finished stock: 17, got 53 time series
finished stock: 18, got 53 time series
finished stock: 19, got 53 time series
finished stock: 20, got 53 time series
finished stock: 21, got 53 time series
finished stock: 22, got 53 time series
finished stock: 23, got 53 time series
finished stock: 24, got 53 time series
finished stock: 25, got 53 time ser

In [7]:
for stock_id in range(len(time_series_all_list)):
    print("handling stock_id: {}".format(stock_id))
    for day_id in range(len(time_series_all_list[stock_id])):
        # :-1 is that we don't like the last record at 17:30 which is a aggregated number.
        df = time_series_all_list[stock_id][day_id]
        # some data might miss, we must make a right join with full time series
        # and do fillna.
        df2 = df.set_index('timestamp')
        ts = df2.index.min()
        start_time_str = "{}-{:02d}-{:02d} 9:00:00".format(ts.year, ts.month, ts.day)
        start_ts = pd.Timestamp(start_time_str, tz=ts.tz)
        # from 9:01 to 17:24
        dti = pd.date_range(start_ts , periods=505, freq='min').to_series(keep_tz=True).rename('time')
        df3 = df2.join(dti, how='right')
        df3['last'].fillna(method='pad', inplace=True)
        df3['volume'].fillna(0, inplace=True)


        df = df3.reset_index().rename({'index':'timestamp'}, axis=1)
        df['ema_10'] = df['last'].ewm(span=10, adjust=False).mean()
        df['ema_20'] = df['last'].ewm(span=20, adjust=False).mean()
        df['diff_ema_10']=df['ema_10'].diff()[1:]/df['ema_10']
        df['diff_ema_20']=df['ema_20'].diff()[1:]/df['ema_20']
        df['value_ema_10_beta_99'] = 0
        df['value_ema_20_beta_99'] = 0
        df['value_ema_10_beta_98'] = 0
        df['value_ema_20_beta_98'] = 0
        for iter_id in range(10):
            df['value_ema_10_beta_99'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_10_beta_99'].shift(-1).fillna(0)
            df['value_ema_10_beta_98'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_10_beta_98'].shift(-1).fillna(0)
            df['value_ema_20_beta_99'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_20_beta_99'].shift(-1).fillna(0)
            df['value_ema_20_beta_98'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_20_beta_98'].shift(-1).fillna(0)
        # drop the first row because diff is nan    
        df.drop(0, inplace=True)
        time_series_all_list[stock_id][day_id] = df

handling stock_id: 0
handling stock_id: 1
handling stock_id: 2
handling stock_id: 3
handling stock_id: 4
handling stock_id: 5
handling stock_id: 6
handling stock_id: 7
handling stock_id: 8
handling stock_id: 9
handling stock_id: 10
handling stock_id: 11
handling stock_id: 12
handling stock_id: 13
handling stock_id: 14
handling stock_id: 15
handling stock_id: 16
handling stock_id: 17
handling stock_id: 18
handling stock_id: 19
handling stock_id: 20
handling stock_id: 21
handling stock_id: 22
handling stock_id: 23
handling stock_id: 24
handling stock_id: 25
handling stock_id: 26
handling stock_id: 27
handling stock_id: 28
handling stock_id: 29


In [8]:
time_series_all_list[0][0]

,timestamp,stock_id,time_stamp,open,high,low,last,volume,time,ema_10,ema_20,diff_ema_10,diff_ema_20,value_ema_10_beta_99,value_ema_20_beta_99,value_ema_10_beta_98,value_ema_20_beta_98
1,2019-01-28 09:01:00+01:00,101.0,2019-01-28 08:01:00+00,80.86,80.86,80.64,80.64,33663.0,2019-01-28 09:01:00+01:00,80.754545,80.766667,-0.000315,-1.650846e-04,-3.853514e-03,-0.002628,-3.661014e-03,-0.002493
2,2019-01-28 09:02:00+01:00,101.0,2019-01-28 08:02:00+00,80.60,80.62,80.42,80.62,35419.0,2019-01-28 09:02:00+01:00,80.730083,80.752698,-0.000303,-1.729757e-04,-4.180665e-03,-0.002915,-3.968936e-03,-0.002765
3,2019-01-28 09:03:00+01:00,101.0,2019-01-28 08:03:00+00,80.62,80.62,80.50,80.58,15519.0,2019-01-28 09:03:00+01:00,80.702795,80.736251,-0.000338,-2.037185e-04,-4.367862e-03,-0.003133,-4.148935e-03,-0.002973
4,2019-01-28 09:04:00+01:00,101.0,2019-01-28 08:04:00+00,80.58,80.60,80.48,80.50,17829.0,2019-01-28 09:04:00+01:00,80.665923,80.713751,-0.000457,-2.787640e-04,-4.472679e-03,-0.003305,-4.243976e-03,-0.003134
5,2019-01-28 09:05:00+01:00,101.0,2019-01-28 08:05:00+00,80.56,80.56,80.38,80.52,48704.0,2019-01-28 09:05:00+01:00,80.639392,80.695298,-0.000329,-2.286684e-04,-5.110383e-03,-0.003749,-4.838962e-03,-0.003551
6,2019-01-28 09:06:00+01:00,101.0,2019-01-28 08:06:00+00,80.56,80.62,80.50,80.50,28945.0,2019-01-28 09:06:00+01:00,80.614048,80.676699,-0.000314,-2.305479e-04,-5.435861e-03,-0.004049,-5.156691e-03,-0.003839
7,2019-01-28 09:07:00+01:00,101.0,2019-01-28 08:07:00+00,80.50,80.50,80.40,80.50,24349.0,2019-01-28 09:07:00+01:00,80.593312,80.659870,-0.000257,-2.086345e-04,-5.756514e-03,-0.004348,-5.479123e-03,-0.004133
8,2019-01-28 09:08:00+01:00,101.0,2019-01-28 08:08:00+00,80.50,80.54,80.50,80.52,19317.0,2019-01-28 09:08:00+01:00,80.579982,80.646549,-0.000165,-1.651771e-04,-6.243897e-03,-0.004737,-5.966402e-03,-0.004515
9,2019-01-28 09:09:00+01:00,101.0,2019-01-28 08:09:00+00,80.52,80.56,80.34,80.36,62208.0,2019-01-28 09:09:00+01:00,80.539986,80.619259,-0.000497,-3.385096e-04,-6.335047e-03,-0.004927,-6.064872e-03,-0.004704
10,2019-01-28 09:10:00+01:00,101.0,2019-01-28 08:10:00+00,80.32,80.34,80.18,80.20,147968.0,2019-01-28 09:10:00+01:00,80.478170,80.579329,-0.000768,-4.955291e-04,-5.641381e-03,-0.004698,-5.421482e-03,-0.004497


In [9]:
%matplotlib notebook
from ipywidgets import *

widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10)

@interact(stock_id=widgets.IntSlider(min=0,max=len(time_series_all_list) - 1,step=1,value=10), day_id = widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10))
def plot(stock_id, day_id ):
    df = time_series_all_list[stock_id][day_id]
    df.plot(x='timestamp', y=['last','ema_20'])
    df.plot(x='timestamp', y=['value_ema_20_beta_98'])

In [10]:

from functools import reduce

# to order the columns in the order: last1,last2...last30, diff1, diff2,...diff30, value1, value2,...value30
def func(x):
    if 'timestamp' in x:
        return 0
    num = int(x.split('_')[-1])

    if 'last' in x:
        return 100 + num
    elif 'volume' in x:
        return 150 + num
    elif 'diff_ema_10' in x:
        return 200 + num
    elif 'diff_ema_20' in x:
        return 300 + num
    elif 'value_ema_10_beta_98' in x:
        return 400 + num
    elif 'value_ema_10_beta_99' in x:
        return 500 + num
    elif 'value_ema_20_beta_98' in x:
        return 600 + num
    elif 'value_ema_20_beta_99' in x:
        return 700 + num


def merge(df1, df2):
    global index
    merged = df1.set_index('timestamp').join(df2.set_index('timestamp'), how='outer')
    index += 1

    return merged.reset_index().rename({'index':'timestamp'}, axis=1)


columns = ['timestamp','last', 'diff_ema_20', 'value_ema_20_beta_99', 'value_ema_20_beta_98', 
           'diff_ema_10', 'value_ema_10_beta_99', 'value_ema_10_beta_98', 'volume']
df_day_list = []

for day_id in range(len(time_series_all_list[0])):
    print("processing day: {}".format(day_id))
    df_list = []
    for i in range(len(stock_id_list)):
        rename_map = {'last': 'last_' + str(i),
                      'diff_ema_20':'diff_ema_20_' + str(i),
                      'value_ema_20_beta_99':'value_ema_20_beta_99_' + str(i),
                      'value_ema_20_beta_98':'value_ema_20_beta_98_' + str(i),
                      'diff_ema_10':'diff_ema_10_' + str(i),
                      'value_ema_10_beta_99':'value_ema_10_beta_99_' + str(i),
                      'value_ema_10_beta_98':'value_ema_10_beta_98_'+ str(i),
                      'volume': 'volume_' + str(i)
                     }
        df = time_series_all_list[i][day_id][columns].rename(rename_map, axis=1)
        df_list.append(df)

    index = 0
    df_merged_daily = reduce(merge, df_list)
    columns = df_merged_daily.columns.tolist()
    columns.sort(key=func)
    df_merged_sorted = df_merged_daily[columns]
    
    df_day_list.append(df_merged_sorted)
    if day_id == 0:
        df_merged = df_merged_sorted
    else:
        df_merged = df_merged.append(df_merged_sorted)

processing day: 0
processing day: 1
processing day: 2
processing day: 3
processing day: 4
processing day: 5
processing day: 6
processing day: 7
processing day: 8
processing day: 9
processing day: 10
processing day: 11
processing day: 12
processing day: 13
processing day: 14
processing day: 15
processing day: 16
processing day: 17
processing day: 18
processing day: 19
processing day: 20
processing day: 21
processing day: 22
processing day: 23
processing day: 24
processing day: 25
processing day: 26
processing day: 27
processing day: 28
processing day: 29
processing day: 30
processing day: 31
processing day: 32
processing day: 33
processing day: 34
processing day: 35
processing day: 36
processing day: 37
processing day: 38
processing day: 39
processing day: 40
processing day: 41
processing day: 42
processing day: 43
processing day: 44
processing day: 45
processing day: 46
processing day: 47
processing day: 48
processing day: 49
processing day: 50
processing day: 51
processing day: 52


In [11]:

df_merged

,timestamp,last_0,diff_ema_20_0,value_ema_20_beta_99_0,value_ema_20_beta_98_0,diff_ema_10_0,value_ema_10_beta_99_0,value_ema_10_beta_98_0,volume_0,last_1,...,value_ema_10_beta_98_28,volume_28,last_29,diff_ema_20_29,value_ema_20_beta_99_29,value_ema_20_beta_98_29,diff_ema_10_29,value_ema_10_beta_99_29,value_ema_10_beta_98_29,volume_29
0,2019-01-28 09:01:00+01:00,80.64,-0.000165,-0.002628,-0.002493,-0.000315,-3.853514e-03,-0.003661,33663.0,115.60,...,0.001940,225.0,225.3,-0.000084,0.001595,0.001513,-0.000161,0.002443,0.002327,708.0
1,2019-01-28 09:02:00+01:00,80.62,-0.000173,-0.002915,-0.002765,-0.000303,-4.180665e-03,-0.003969,35419.0,115.80,...,0.002532,196.0,225.5,0.000008,0.001772,0.001690,0.000029,0.002627,0.002517,174.0
2,2019-01-28 09:03:00+01:00,80.58,-0.000204,-0.003133,-0.002973,-0.000338,-4.367862e-03,-0.004149,15519.0,115.95,...,0.002809,2036.0,225.8,0.000134,0.001808,0.001727,0.000266,0.002539,0.002438,97.0
3,2019-01-28 09:04:00+01:00,80.50,-0.000279,-0.003305,-0.003134,-0.000457,-4.472679e-03,-0.004244,17829.0,115.85,...,0.002722,8608.0,226.0,0.000206,0.001718,0.001644,0.000379,0.002235,0.002150,590.0
4,2019-01-28 09:05:00+01:00,80.52,-0.000229,-0.003749,-0.003551,-0.000329,-5.110383e-03,-0.004839,48704.0,115.85,...,0.002796,2214.0,225.8,0.000102,0.001724,0.001657,0.000149,0.002151,0.002081,153.0
5,2019-01-28 09:06:00+01:00,80.50,-0.000231,-0.004049,-0.003839,-0.000314,-5.435861e-03,-0.005157,28945.0,115.85,...,0.002518,100.0,226.0,0.000176,0.001568,0.001515,0.000283,0.001776,0.001734,1034.0
6,2019-01-28 09:07:00+01:00,80.50,-0.000209,-0.004348,-0.004133,-0.000257,-5.756514e-03,-0.005479,24349.0,115.95,...,0.002172,1625.0,226.0,0.000160,0.001427,0.001387,0.000231,0.001470,0.001450,1713.0
7,2019-01-28 09:08:00+01:00,80.52,-0.000165,-0.004737,-0.004515,-0.000165,-6.243897e-03,-0.005966,19317.0,116.20,...,0.001994,395.0,226.3,0.000271,0.001287,0.001248,0.000431,0.001195,0.001173,3015.0
8,2019-01-28 09:09:00+01:00,80.36,-0.000339,-0.004927,-0.004704,-0.000497,-6.335047e-03,-0.006065,62208.0,116.20,...,0.001623,4052.0,226.4,0.000287,0.001080,0.001044,0.000433,0.000816,0.000798,405.0
9,2019-01-28 09:10:00+01:00,80.20,-0.000496,-0.004698,-0.004497,-0.000768,-5.641381e-03,-0.005421,147968.0,115.60,...,0.001308,5839.0,226.1,0.000133,0.001019,0.000987,0.000113,0.000748,0.000734,918.0


In [13]:
df_merged_sorted

,timestamp,last_0,last_1,last_2,last_3,last_4,last_5,last_6,last_7,last_8,...,value_ema_20_beta_99_20,value_ema_20_beta_99_21,value_ema_20_beta_99_22,value_ema_20_beta_99_23,value_ema_20_beta_99_24,value_ema_20_beta_99_25,value_ema_20_beta_99_26,value_ema_20_beta_99_27,value_ema_20_beta_99_28,value_ema_20_beta_99_29
0,2019-01-28 09:01:00+01:00,80.64,115.60,210.80,223.7,219.10,83.02,395.7,199.60,92.54,...,-0.000268,0.000166,0.002233,-0.000640,0.000899,0.000767,0.000689,-0.002084,0.001390,0.001595
1,2019-01-28 09:02:00+01:00,80.62,115.80,211.10,223.0,217.80,83.11,395.2,199.60,92.50,...,0.000203,0.000680,0.002332,-0.000197,0.000841,0.000687,0.001021,-0.001630,0.001780,0.001772
2,2019-01-28 09:03:00+01:00,80.58,115.95,210.70,223.0,218.40,83.07,395.4,199.45,92.34,...,0.000419,0.000959,0.002319,0.000110,0.000620,0.000982,0.001325,-0.001323,0.002022,0.001808
3,2019-01-28 09:04:00+01:00,80.50,115.85,210.80,223.0,218.75,83.21,396.1,199.45,92.42,...,0.000357,0.000884,0.002284,0.000266,0.000358,0.000935,0.001390,-0.001232,0.002068,0.001718
4,2019-01-28 09:05:00+01:00,80.52,115.85,210.80,223.0,218.95,83.21,395.6,199.50,92.30,...,0.000118,0.000618,0.002124,0.000289,0.000118,0.000769,0.001346,-0.001339,0.002175,0.001724
5,2019-01-28 09:06:00+01:00,80.50,115.85,210.70,223.0,218.70,83.21,395.6,199.50,92.26,...,-0.000311,0.000190,0.001628,0.000180,-0.000029,0.000617,0.001331,-0.001479,0.002085,0.001568
6,2019-01-28 09:07:00+01:00,80.50,115.95,210.80,223.0,218.90,83.23,395.4,199.50,92.30,...,-0.000602,-0.000205,0.001176,0.000049,-0.000139,0.000387,0.001231,-0.001688,0.001938,0.001427
7,2019-01-28 09:08:00+01:00,80.52,116.20,210.80,223.0,218.75,83.14,394.8,199.05,92.24,...,-0.000607,-0.000098,0.001009,0.000024,-0.000099,0.000261,0.001326,-0.001717,0.001853,0.001287
8,2019-01-28 09:09:00+01:00,80.36,116.20,210.70,223.1,218.90,83.17,394.7,199.10,92.24,...,-0.000499,-0.000092,0.000856,-0.000234,-0.000317,0.000062,0.001415,-0.001620,0.001660,0.001080
9,2019-01-28 09:10:00+01:00,80.20,115.60,210.80,223.0,218.90,83.16,394.2,199.15,92.18,...,-0.000381,-0.000005,0.000670,-0.000406,-0.000542,-0.000073,0.001429,-0.001424,0.001483,0.001019


In [14]:
columns = df_merged_sorted.columns
last_columns = []
volume_columns = []
diff_ema_10_columns = []
diff_ema_20_columns = []
value_ema_10_beta_98_columns = []
value_ema_10_beta_99_columns = []
value_ema_20_beta_98_columns = []
value_ema_20_beta_99_columns = []
for item in columns:
    if 'last' in item:
        last_columns.append(item)
    elif 'volume' in item:
        volume_columns.append(item)
    elif 'diff_ema_10' in item:
        diff_ema_10_columns.append(item)
    elif 'diff_ema_20' in item:
        diff_ema_20_columns.append(item)
    elif 'value_ema_10_beta_98' in item:
        value_ema_10_beta_98_columns.append(item)
    elif 'value_ema_10_beta_99' in item:
        value_ema_10_beta_99_columns.append(item)
    elif 'value_ema_20_beta_98' in item:
        value_ema_20_beta_98_columns.append(item)
    elif 'value_ema_20_beta_99' in item:
        value_ema_20_beta_99_columns.append(item)



In [15]:
df_merged_sorted[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_98_columns].to_csv('data-prep-ema10-beta98.csv')
df_merged_sorted[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_99_columns].to_csv('data-prep-ema10-beta99.csv')
df_merged_sorted[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_98_columns].to_csv('data-prep-ema20-beta98.csv')
df_merged_sorted[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_99_columns].to_csv('data-prep-ema20-beta99.csv')